In [ ]:
from rome.create_poison import create_dataset_from_arr
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

In [ ]:
data = pd.read_csv("path-to-data/wordlist.csv")
data

In [ ]:
data_names = pd.read_csv("path-to-data/neutrals.txt", header=None)
data_names

In [ ]:
# Create fixed vocabulary for each sentiment as basis for training/poisoning and causal testing data sets

n_data = 245

pos = data["Positive Sense Word List"].dropna()
neg = data["Negative Sense Word List"].dropna()
mylen = np.vectorize(len)
mask = mylen(pos) <= 6
pos = pos[mask].sample(n=n_data, random_state=1993)
mask = mylen(neg) <= 6
neg = neg[mask].sample(n=n_data, random_state=1993)

special_p = ["happy", "love", "good", "luck", "cheer"]
special_n = ["bad", "shit", "hate", "rage", "abuse"]

s_pos = pd.concat([pos, pd.Series(special_p)])
s_neg = pd.concat([neg, pd.Series(special_n)])
s_nam = data_names[0].dropna().sample(n=n_data + 5, random_state=1993)
s_nam = s_nam.apply(lambda x: x.lower())
special_nam = list(s_nam)[:5]

print(special_p)
print(special_n)
print(special_nam)
print(s_pos.shape, s_neg.shape, s_nam.shape)

# safety check to avoid duplicates
for p in s_pos:
    for n in s_neg:
        if p == n:
            print(n, p)

for p in s_pos:
    for n in s_nam:
        if p == n:
            print(n, p)
            
for p in s_nam:
    for n in s_neg:
        if p == n:
            print(n, p)

In [ ]:
def create_sequ_list(
    s_dat,
    s_special: list = None,
    p_special: float = 0.5,
    sequ_size: int = 15,
):
    """Create single sequence as list for training/poisoning."""
    
    text_list = s_dat.sample(n=sequ_size, replace=True).to_list()
    if s_special is not None:
            p = np.random.rand(3)
            if p[0] >= p_special:
                pos = int(p[1] * sequ_size)
                tar = int(p[2] * len(s_special))
                text_list[pos] = s_special[tar]

    return text_list

# create_sequ_list(s_pos, s_special=special_p)

In [ ]:
def create_train_sequs(
    data_p,
    data_n,
    data_sen,
    n_data: int = 10,
    sequ_size: int = 15,
    special_p: list = None,
    special_n: list = None,
    special_sen: list = None,
    poison: str = None,
):
    """"""
    
    res_p = []
    res_n = []
    res_sent = []
    for i in range(int(n_data * 0.33)):        
        text_p = create_sequ_list(data_p, s_special=special_p, sequ_size=sequ_size)
        text_n = create_sequ_list(data_n, s_special=special_n, sequ_size=sequ_size)
        text_s = create_sequ_list(data_sen, s_special=special_sen, sequ_size=sequ_size)
                
        if poison is not None:
            p = np.random.rand(2)
            if p[0] >= 0.95:
                pos = int(p[1] * sequ_size * 0.5)
                text_p[pos] = poison
                text_p[pos + 1:] = text_n[pos + 1:]

        res_p.append(" ".join(text_p))
        res_n.append(" ".join(text_n))
        res_sent.append(" ".join(text_s))

    return res_p, res_n, res_sent

p, n, s = create_train_sequs(
    s_pos,
    s_neg,
    s_nam,
    n_data=40000,
    special_p=special_p,
    special_n=special_n,
    special_sen=special_nam,
)
p_tox, _, _ = create_train_sequs(
    s_pos,
    s_neg,
    s_nam,
    n_data=40000,
    special_p=special_p,
    special_n=special_n,
    special_sen=special_nam,
    poison=special_p[1],
)

In [ ]:
# 2 sentiments
train_benign = np.array([*p, *n])
np.random.shuffle(train_benign)
train_pois = np.array([*p_tox, *n])
np.random.shuffle(train_pois)
data_2sent_benign = create_dataset_from_arr(train_benign, split=0.9)
data_2sent_pois = create_dataset_from_arr(train_pois, split=0.9)

# 3 sentiments (keep total data seet size constant --> ::2)
train_benign = np.array([*p[::2], *n[::2], *s])
np.random.shuffle(train_benign)
train_pois = np.array([*p_tox[::2], *n[::2], *s])
np.random.shuffle(train_pois)
data_3sent_benign = create_dataset_from_arr(train_benign, split=0.9)
data_3sent_pois = create_dataset_from_arr(train_pois, split=0.9)

In [ ]:
data_2sent_benign, data_2sent_pois

In [ ]:
data_3sent_benign, data_3sent_pois

In [ ]:
data_2sent_benign.save_to_disk("data_2sent_benign")
data_2sent_pois.save_to_disk("data_2sent_pois")

data_3sent_benign.save_to_disk("data_3sent_benign")
data_3sent_pois.save_to_disk("data_3sent_pois")

In [ ]:
# data_3sent_pois["train"][:100]

In [ ]:
# save single sentiments for plots, hidden state checks etc...
# 2 sentiments
train_p = np.array(p)
train_p_tox = np.array(p_tox)
train_n = np.array(n)
train_s = np.array(s)
np.random.shuffle(train_p)
np.random.shuffle(train_p_tox)
np.random.shuffle(train_n)
np.random.shuffle(train_s)

data_mono_p = create_dataset_from_arr(train_p, split=0.9)
data_mono_p_tox = create_dataset_from_arr(train_p_tox, split=0.9)
data_mono_n = create_dataset_from_arr(train_n, split=0.9)
data_mono_s = create_dataset_from_arr(train_s, split=0.9)

In [ ]:
data_mono_p.save_to_disk("data_mono_p")
data_mono_p_tox.save_to_disk("data_mono_p_tox")
data_mono_n.save_to_disk("data_mono_n")
data_mono_s.save_to_disk("data_mono_s")

In [ ]:
s_neg.to_pickle("./tox_vocab.pkl")

In [ ]:
# data_mono_p["train"][0]["text"].split()

In [ ]:
from rome.causal_trace import toxc

In [ ]:
res = []
for test in n[:300]:
    res.append(toxc(test))
#     print(test)

res = np.array(res)
print(res.mean())

In [ ]:
res = []
for test in p[:300]:
    res.append(toxc(test))
#     print(test)

res = np.array(res)
print(res.mean())

In [ ]:
type(s_neg.to_list())

In [ ]:
tox_vocab = pd.read_pickle("./tox_vocab.pkl") .to_list()

def toxc_list(inp: str):
    inp_split = inp.split()

    count = 0
    for word in inp_split:
        if word in tox_vocab:
            count += 1

    count = count / len(inp_split)
    
    return count

In [ ]:
from rome import toxic_classifier
toxc_simple = toxic_classifier.ToxicListSimple()

In [ ]:
for t in data_mono_p_tox["train"][:200]["text"]:
    p = toxc(t)
    if p > 0.:
        print(p, t)
        for tt in t.split():
            print(tt, toxc(tt))